In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
# Read yelp business data into dataframe
unfilteredBusinessData=pd.read_json("data/yelp_academic_dataset_business.json", lines=True)

# unfilteredReviews=pd.read_json("yelp_academic_dataset_review.json", lines=True)

In [4]:
# Filter business data
# unfilteredData.info()

# Select all businesses with "Restaurants" category
restaurants_df = unfilteredBusinessData[(unfilteredBusinessData["categories"].str.contains("restaurants", case=False, na=False))]

# Select all restaurants with > 30 reviews
reviewed_restaurants =  restaurants_df[(restaurants_df["review_count"] >= 30)]

reviewed_restaurants.head()


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
14,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants","{'Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-..."
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
20,WKMJwqnfZKsAae75RMP6jA,Roast Coffeehouse and Wine Bar,10359 104 Street NW,Edmonton,AB,T5J 1B9,53.546045,-113.499169,4.0,40,0,"{'OutdoorSeating': 'False', 'Caters': 'True', ...","Coffee & Tea, Food, Cafes, Bars, Wine Bars, Re...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."


In [4]:
# Read yelp user data into dataframe
unfilteredUsers=pd.DataFrame()

userReviewReader=pd.read_json("data/yelp_academic_dataset_user.json", lines=True, chunksize=100000)
for chunk in userReviewReader:
    unfilteredUsers = pd.concat([unfilteredUsers, chunk])

In [5]:
# Examine user data
unfilteredUsers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 333.7+ MB


In [5]:
# Read yelp review data info dataframe
unfilteredReviews = pd.DataFrame()
reviewReader=pd.read_json("data/yelp_academic_dataset_review.json", lines=True, chunksize=100000)
for chunk in reviewReader:
    unfilteredReviews = pd.concat([unfilteredReviews, chunk])

In [18]:
# Examine review data
# unfilteredReviews.info()

# Select only reviews of businesses in our filtered dataset
reviewed_restaurant_mask = unfilteredReviews['business_id'].isin(reviewed_restaurants['business_id'])
filtered_reviews = unfilteredReviews[reviewed_restaurant_mask]
filtered_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4371248 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 333.5+ MB
